In [ ]:
import cv2             #opencv library
import numpy as np     #numerical python 

#state intended ASCII font size for your application
#treat ascii as 6pt font - clearer, larger images!
FONT_SIZE=6 #12

AFS_HEIGHT = 16 if FONT_SIZE == 6 else 34
AFS_WIDTH = 8 if FONT_SIZE == 6 else 17
RESOURCE_PATH = './Resources/ASCII-6pt/' if FONT_SIZE == 6 else './Resources/ASCII-12pt/'

ref = '/.|{}[]?><!@#$%^&*()_-~`\'\\¶§ღ' #list of characters to reference
ASCII_dict = np.zeros((len(ref),AFS_HEIGHT,AFS_WIDTH),np.uint8)#3d array of all matrixes for each character

def initASCII():
    for i in range(0,len(ref)):
        kern = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        gray = cv2.imread(RESOURCE_PATH+ str(i+1) + '.png',0)
        closed = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kern)
        ASCII_dict[i]=closed.astype(np.uint8)       

def pattern(matrix):
    most = -1000000
    ASCIIreturn = 0 #defaults 0 (/)
    for c in range(0,len(ref)):
        comp = ASCII_dict[c]
        res = cv2.matchTemplate(matrix,comp,cv2.TM_CCORR_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        
        if(min_val > most):
            most = res
            ASCIIreturn = c
            
    return (most,ASCIIreturn)

def highlight(matrix):
    most = 100000
    ASCIIreturn = 26 #defaults 26(¶)
    for c in range(0,len(ref)):
        comp = ASCII_dict[c]
        res = cv2.matchTemplate(matrix,comp,cv2.TM_SQDIFF_NORMED)
        if(res < most):
            most = res
            ASCIIreturn = c
            
    return (most,ASCIIreturn) 

def shadow(matrix):
    ASCIIreturn = 0 #defaults 0 (/)
    most = 10000
    for c in range(0,len(ref)):
        comp = ASCII_dict[c]
        res = cv2.matchTemplate(matrix,comp,cv2.TM_CCORR)
        if(res < most):
            most = res
            ASCIIreturn = c
            
    return ASCIIreturn 

def createASCIIimage(img):
    board = ["" for x in range(1+image.shape[0]//AFS_HEIGHT)] #initializing a new local board for reference/output
    height = img.shape[0]//AFS_HEIGHT
    width = img.shape[1]//AFS_WIDTH
    for row in range(0,(height)-1):
        for col in range(0,(width)-1):
            conf1,ASCII_val1 = highlight(img[row*AFS_HEIGHT:((row+1)*AFS_HEIGHT), col*AFS_WIDTH:((col+1)*AFS_WIDTH)].astype(np.uint8))
            ASCII_val = ASCII_val1 if conf1 == 1 and ASCII_val1 == 0 else 1
            conf2,ASCII_val2 = pattern(img[row*AFS_HEIGHT:((row+1)*AFS_HEIGHT), col*AFS_WIDTH:((col+1)*AFS_WIDTH)].astype(np.uint8)) 
            ASCII_val = ASCII_val2 if conf2 >= 0.94 else ASCII_val
            ASCII_val3 = shadow(img[row*AFS_HEIGHT:((row+1)*AFS_HEIGHT), col*AFS_WIDTH:((col+1)*AFS_WIDTH)].astype(np.uint8))
            ASCII_val = ASCII_val3 if ASCII_val3 != 0 else ASCII_val
            board[row] += ref[ASCII_val]
    return board
    
def printASCII(art):
    for row in range(0,len(art)):
        x = ""
        for i in art[row]:
            x += i
        print(x)

image = cv2.imread('./images/Mona_Lisa.png',0) #Read in image to convert
initASCII() #Load ASCII character matrix
ASCII_image = createASCIIimage(image) #Perform conversion
printASCII(ASCII_image) #Print ASCII result

